…a pak Hynek Cígler napíše, že čím méně filmů do srovnání bude, tím bude průkaznější. Samozřejmě. Takže se to profiltrujem podle počtu hodnocení na ČSFD a trochu i růčo.

In [19]:
import os
import polars as pl
import altair as alt
import warnings

pl.Config(tbl_rows=100)

df = pl.read_parquet(os.path.join("data","fdb.parquet"))
vanocni = pl.read_json(os.path.join("data","vanocni_filmy_kompletni_metadata.json"))

from src.kristi_promin import kristi_promin
from src.me_to_neurazi import me_to_neurazi
from src.alt_friendly import alt_friendly

pl.Config.set_tbl_rows(150)
pl.Config.set_fmt_str_lengths(150)
pl.Config.set_tbl_width_chars(200)

alt.themes.register('irozhlas', kristi_promin)
alt.themes.enable('irozhlas')
warnings.filterwarnings("ignore")

In [20]:
with open("data_raw/vanocni_filmy_csfd.json", "r", encoding="utf-8") as imp:
    vanocni = json.loads(imp.read())
vanocni = pl.DataFrame(vanocni).transpose().rename({"column_0":"nazev",'column_1':'url','column_2':'format'})
vanocni

nazev,url,format
str,str,str
"""Adéla ještě nevečeřela""","""https://www.csfd.cz/film/5981-adela-jeste-nevecerela/prehled/""","""film"""
"""Anděl Páně""","""https://www.csfd.cz/film/215414-andel-pane/prehled/""","""film"""
"""Ať přiletí čáp, královno!""","""https://www.csfd.cz/film/108859-at-prileti-cap-kralovno/prehled/""","""tv"""
"""Bohouš""","""https://www.csfd.cz/film/8186-bohous/prehled/""","""tv"""
"""Byl jednou jeden král...""","""https://www.csfd.cz/film/10089-byl-jednou-jeden-kral/prehled/""","""film"""
"""Cesta do Ameriky""","""https://www.csfd.cz/film/5577-cesta-do-ameriky/prehled/""","""film"""
"""Co takhle svatba, princi?""","""https://www.csfd.cz/film/108854-co-takhle-svatba-princi/prehled/""","""tv"""
"""Císař a tambor""","""https://www.csfd.cz/film/5345-cisar-a-tambor/prehled/""","""film"""
"""Císařův pekař""","""https://www.csfd.cz/film/3094-cisaruv-pekar-pekaruv-cisar/prehled/""","""film"""


In [21]:
vanocni_detaily = pl.scan_ndjson("data_raw/csfd/*.json").collect().rename({'titul':'nazev'})

vanocni_detaily

id,nazev,zanry,zeme,rok,delka,rezie,scenar,hraji,anotace,hodnoceni,pocet_hodnoceni
str,str,list[str],str,i64,i64,list[str],list[str],list[str],str,i64,i64
"""10086-nesmrtelna-teta""","""Nesmrtelná teta""","[""pohádka""]","""Česko""",1993,98,"[""Zdeněk Zelenka""]","[""Zdeněk Zelenka""]","[""Jiřina Bohdalová"", ""Jaromír Hanzlík"", … ""více""]","""Temperamentní situační pohádková komedie o vesnickém chasníkovi Matějovi, který náhle pobral rozum a rozhodl se usilovat o ruku princezny Pavlínky, je…",63,23249
"""10089-byl-jednou-jeden-kral""","""Byl jednou jeden král...""","[""pohádka"", ""komedie""]","""Československo""",1954,107,"[""Bořivoj Zeman""]","[""Jan Werich"", ""Jiří Brdečka"", ""Bořivoj Zeman""]","[""Jan Werich"", ""Vlasta Burian"", … ""více""]","""Byl jednou jeden král a ten si říkal Já První. Chtěl odevzdat vládu té ze tří dcer, která ho má nejraději. Drahomíra ho má ráda jako zlato, Zpěvanka j…",87,56191
"""10094-pysna-princezna""","""Pyšná princezna""","[""pohádka"", ""komedie"", … ""romantický""]","""Československo""",1952,94,"[""Bořivoj Zeman""]","[""Bořivoj Zeman"", ""Henryk Bloch"", ""Oldřich Kautský""]","[""Alena Vránová"", ""Vladimír Ráž"", … ""více""]","""Pohádka všech českých pohádek o pyšné princezně Krasomile, která odmítla vzít si za muže krále Miroslava. Ten si to však nenechal líbit a v přestrojen…",82,28205
"""10096-silene-smutna-princezna""","""Šíleně smutná princezna""","[""pohádka"", ""hudební"", ""komedie""]","""Československo""",1968,89,"[""Bořivoj Zeman""]","[""František Vlček st."", ""Bořivoj Zeman""]","[""Helena Vondráčková"", ""Václav Neckář"", … ""více""]","""Hudební pohádka Šíleně smutná princezna je určena především malým - ovšem ne těm nejmenším divákům. Vypráví příběh prince a princezny ze sousedních sp…",83,32665
"""108846-o-medvedu-ondrejovi""","""O medvědu Ondřejovi""","[""pohádka"", ""komedie""]","""Československo""",1959,50,"[""Jaroslav Mach""]",null,"[""Jaroslav Marvan"", ""Aglaia Morávková"", … ""více""]","""Princezna Blanka se tajně schází se svým milým, myslivcem Ondřejem. Jejich lásce přeje jen komorná Anežka, královský otec se nesmí nic dozvědět. Blank…",59,1851
"""108854-co-takhle-svatba-princi""","""Co takhle svatba, princi?""","[""pohádka"", ""hudební""]","""Československo""",1986,90,"[""Jiří Adamec""]","[""Jiří Chalupa""]","[""Miroslav Vladyka"", ""Ivana Andrlová"", … ""více""]","""Kdo by neznal jednu z nejznámějších filmových písniček ""Když se načančám"". Tuto a další písničky (např. ""Taková jsem já"", ""Princezny jdou si hajnout"" …",69,7403
"""108859-at-prileti-cap-kralovno""","""Ať přiletí čáp, královno!""","[""pohádka"", ""hudební""]","""Československo""",1988,93,"[""Jiří Adamec""]","[""Jiří Chalupa""]","[""Miroslav Vladyka"", ""Ivana Andrlová"", … ""více""]","""Ve volném pokračování pohádkové komedie Co takhle svatba, princi? se znovu setkáte s princem Davidem a jeho ženou Bělou. Oba se těší na děťátko, které…",61,5954
"""111222-kotva-u-privozu""","""Kotva u přívozu""","[""komedie"", ""drama""]","""Československo""",1980,90,"[""Marie Poledňáková""]","[""Marie Poledňáková""]","[""Jana Šulcová"", ""Petr Haničinec"", … ""více""]","""Magda je televizní hlasatelka. Pro miliony diváků příjemná, sympatická tvář na obrazovce. Klidná, úsměvná, vyrovnaná. Příchod pohody do rodinného veče…",60,2020
"""121859-princove-jsou-na-draka""","""Princové jsou na draka""","[""pohádka"", ""hudební""]","""Československo""",1980,57,"[""Jiří Adamec""]","[""Jiří Chalupa""]","[""Vladimír Menšík"", ""Ivana Andrlová"", … ""více""]","""Vtipný příběh o lásce a moudrosti, z kterého vyplývá, že nad drakem mohou zvítězit nejen princové. (oficiální text distributora)""",74,11814


In [22]:
df.sample(1)

kanal,datum,nazev,extra,noticka,url
cat,datetime[μs],str,str,str,str
"""ČT2""",2011-03-24 12:40:00,"""Pozvánka na Dvojku""",null,null,null


In [23]:
vanocni.sample(1)

nazev,url,format
str,str,str
"""Pyšná princezna""","""https://www.csfd.cz/film/10094-pysna-princezna/prehled/""","""film"""


In [24]:
vanocni_final = vanocni.with_columns(
    pl.col('nazev').replace({'Císařův pekař - Pekařův císař':'Císařův pekař'})
).join(
    vanocni_detaily,
    how='right',
    on='nazev'
).sort(
    by='hodnoceni', descending=True
).filter(
    ~pl.col("url").is_null()
)

In [35]:
vanocni_final.sample()

url,format,id,nazev,zanry,zeme,rok,delka,rezie,scenar,hraji,anotace,hodnoceni,pocet_hodnoceni
str,str,str,str,list[str],str,i64,i64,list[str],list[str],list[str],str,i64,i64
"""https://www.csfd.cz/film/5335-silvestr-svobodneho-pana/prehled/""","""tv""","""5335-silvestr-svobodneho-pana""","""Silvestr svobodného pána""","[""komedie""]","""Československo""",1979,31,"[""Jiří Krejčík""]","[""Jiří Krejčík"", ""Gustav Oplustil""]","[""Josef Abrhám"", ""Libuše Šafránková"", … ""Jana Vaňková""]","""Petr, mladý inženýr, má rozličné pletky s dívkami, od nedospělé Milušky v domě, kde žije, až po další, i vdané paničky. Chystá se zrovna na Silvestra,…",61,887


In [37]:
vanocni_final.sort(by='pocet_hodnoceni')

url,format,id,nazev,zanry,zeme,rok,delka,rezie,scenar,hraji,anotace,hodnoceni,pocet_hodnoceni
str,str,str,str,list[str],str,i64,i64,list[str],list[str],list[str],str,i64,i64
"""https://www.csfd.cz/film/5335-silvestr-svobodneho-pana/prehled/""","""tv""","""5335-silvestr-svobodneho-pana""","""Silvestr svobodného pána""","[""komedie""]","""Československo""",1979,31,"[""Jiří Krejčík""]","[""Jiří Krejčík"", ""Gustav Oplustil""]","[""Josef Abrhám"", ""Libuše Šafránková"", … ""Jana Vaňková""]","""Petr, mladý inženýr, má rozličné pletky s dívkami, od nedospělé Milušky v domě, kde žije, až po další, i vdané paničky. Chystá se zrovna na Silvestra,…",61,887
"""https://www.csfd.cz/film/22216-dunston-sam-v-hotelu/prehled/""","""film""","""22216-dunston-sam-v-hotelu""","""Dunston: Sám v hotelu""","[""komedie"", ""dobrodružný"", ""rodinný""]","""USA""",1996,85,"[""Ken Kwapis""]","[""John Hopkins"", ""Bruce Graham""]","[""Jason Alexander"", ""Faye Dunaway"", … ""více""]","""Nepřejte si vědět, co se stane, když se v pětihvězdičkovém hotelu objeví zloděj šperků se svým orangutanem. Ve chvíli, kdy se navíc čilá opička pánovi…",51,1132
"""https://www.csfd.cz/film/236348-o-kominickem-ucni-a-dceri-cukrare/prehled/""","""film""","""236348-o-kominickem-ucni-a-dceri-cukrare""","""O kominickém učni a dceři cukráře""","[""rodinný"", ""romantický""]","""Česko""",2007,63,"[""Pavel Jandourek""]","[""Václav Holanec"", ""Alena Bartošíková""]","[""Tomáš Materna"", ""Adriana Neubauerová"", … ""více""]","""Sirotek Tomáš (T. Materna) musel nastoupit do učení ke kominickému mistru Dýmalovi (P. Zedníček) a podřídit se tak rozhodnutí svého strýce. Začátky ne…",72,1763
"""https://www.csfd.cz/film/108846-o-medvedu-ondrejovi/prehled/""","""film""","""108846-o-medvedu-ondrejovi""","""O medvědu Ondřejovi""","[""pohádka"", ""komedie""]","""Československo""",1959,50,"[""Jaroslav Mach""]",null,"[""Jaroslav Marvan"", ""Aglaia Morávková"", … ""více""]","""Princezna Blanka se tajně schází se svým milým, myslivcem Ondřejem. Jejich lásce přeje jen komorná Anežka, královský otec se nesmí nic dozvědět. Blank…",59,1851
"""https://www.csfd.cz/film/111222-kotva-u-privozu/prehled/""","""tv""","""111222-kotva-u-privozu""","""Kotva u přívozu""","[""komedie"", ""drama""]","""Československo""",1980,90,"[""Marie Poledňáková""]","[""Marie Poledňáková""]","[""Jana Šulcová"", ""Petr Haničinec"", … ""více""]","""Magda je televizní hlasatelka. Pro miliony diváků příjemná, sympatická tvář na obrazovce. Klidná, úsměvná, vyrovnaná. Příchod pohody do rodinného veče…",60,2020
"""https://www.csfd.cz/film/3118-kral-kralu/prehled/""","""film""","""3118-kral-kralu""","""Král Králů""","[""komedie""]","""Československo""",1963,100,"[""Martin Frič""]","[""Jiří Mucha""]","[""Jiří Sovák"", ""Miloš Kopecký"", … ""více""]","""Uprostřed pouště nalezne osamělý jezdec na oslu starého muže, který leží v bezvědomí ve stínu luxusního automobilu. Jezdec naloží starce na osla a odv…",65,2645
"""https://www.csfd.cz/film/197358-dalskabaty-hrisna-ves-aneb-zapomenuty-cert/prehled/""","""tv""","""197358-dalskabaty-hrisna-ves-aneb-zapomenuty-cert""","""Dalskabáty, hříšná ves aneb Zapomenutý čert""","[""pohádka"", ""komedie""]","""Československo""",1976,116,"[""Jaroslav Novotný""]","[""Jan Drda"", ""Jaroslav Novotný""]","[""Jiřina Bohdalová"", ""Jaroslav Moučka"", … ""více""]","""Hubatá, rázná Plajznerka (J. Bohdalová), jíž hraje v těle snad každá žilka touhou po hutném krajíci štěstí, krotí a cepuje zapomenutého čerta, aby z n…",80,3101
"""https://www.csfd.cz/film/5345-cisar-a-tambor/prehled/""","""film""","""5345-cisar-a-tambor""","""Císař a tambor""","[""pohádka""]","""Česko""",1998,76,"[""Václav Křístek""]","[""Václav Křístek""]","[""Ondřej Vetchý"", ""Petr Hradil"", … ""více""]","""Filmová pohádka na motivy pověstí o císaři Josefovi II. Stejně jako on i filmový císař Maxmilián (O. Vetchý) často mizí v přestrojení ze zámku, aby po…",60,3992
"""https://www.csfd.cz/film/344932-duch-nad-zlato/prehled/""","""film""","""3449

In [41]:
vanocni_final.filter(pl.col('pocet_hodnoceni') < 26000)

url,format,id,nazev,zanry,zeme,rok,delka,rezie,scenar,hraji,anotace,hodnoceni,pocet_hodnoceni
str,str,str,str,list[str],str,i64,i64,list[str],list[str],list[str],str,i64,i64
"""https://www.csfd.cz/film/8186-bohous/prehled/""","""tv""","""8186-bohous""","""Bohouš""","[""krátkometrážní"", ""komedie""]","""Československo""",1968,25,"[""Petr Schulhoff""]","[""František Vlček st.""]","[""Jiří Sovák"", ""Vladimír Menšík"", … ""pes Baldur""]","""Správce horského hotýlku Alois Randa (Jiří Sovák) má starostí nad hlavu: očekává početný zájezd turistů, nestíhá vařit a jeho jediná zaměstnankyně, po…",88,7462
"""https://www.csfd.cz/film/197358-dalskabaty-hrisna-ves-aneb-zapomenuty-cert/prehled/""","""tv""","""197358-dalskabaty-hrisna-ves-aneb-zapomenuty-cert""","""Dalskabáty, hříšná ves aneb Zapomenutý čert""","[""pohádka"", ""komedie""]","""Československo""",1976,116,"[""Jaroslav Novotný""]","[""Jan Drda"", ""Jaroslav Novotný""]","[""Jiřina Bohdalová"", ""Jaroslav Moučka"", … ""více""]","""Hubatá, rázná Plajznerka (J. Bohdalová), jíž hraje v těle snad každá žilka touhou po hutném krajíci štěstí, krotí a cepuje zapomenutého čerta, aby z n…",80,3101
"""https://www.csfd.cz/film/4238-panna-a-netvor/prehled/""","""film""","""4238-panna-a-netvor""","""Panna a netvor""","[""pohádka"", ""horor"", … ""romantický""]","""Československo""",1978,88,"[""Juraj Herz""]","[""Juraj Herz"", ""Ota Hofman"", ""František Hrubín""]","[""Zdena Studenková"", ""Vlastimil Harapes"", … ""více""]","""Slavnou strašidelnou pohádku, původně francouzskou, o princi začarovaném do děsivé zvířecí podoby, jehož vysvobodí až čistá láska, využil režisér Jura…",78,10311
"""https://www.csfd.cz/film/5577-cesta-do-ameriky/prehled/""","""film""","""5577-cesta-do-ameriky""","""Cesta do Ameriky""","[""komedie"", ""romantický""]","""USA""",1988,116,"[""John Landis""]","[""David Sheffield"", ""Barry W. Blaustein""]","[""Eddie Murphy"", ""Arsenio Hall"", … ""více""]","""Král komedie Eddie Murphy se v tomto zábavném filmu stává zhýčkaným a sebevědomým africkým princem, který přichází do Ameriky, aby si tu našel nevěstu…",76,23266
"""https://www.csfd.cz/film/121859-princove-jsou-na-draka/prehled/""","""tv""","""121859-princove-jsou-na-draka""","""Princové jsou na draka""","[""pohádka"", ""hudební""]","""Československo""",1980,57,"[""Jiří Adamec""]","[""Jiří Chalupa""]","[""Vladimír Menšík"", ""Ivana Andrlová"", … ""více""]","""Vtipný příběh o lásce a moudrosti, z kterého vyplývá, že nad drakem mohou zvítězit nejen princové. (oficiální text distributora)""",74,11814
"""https://www.csfd.cz/film/236348-o-kominickem-ucni-a-dceri-cukrare/prehled/""","""film""","""236348-o-kominickem-ucni-a-dceri-cukrare""","""O kominickém učni a dceři cukráře""","[""rodinný"", ""romantický""]","""Česko""",2007,63,"[""Pavel Jandourek""]","[""Václav Holanec"", ""Alena Bartošíková""]","[""Tomáš Materna"", ""Adriana Neubauerová"", … ""více""]","""Sirotek Tomáš (T. Materna) musel nastoupit do učení ke kominickému mistru Dýmalovi (P. Zedníček) a podřídit se tak rozhodnutí svého strýce. Začátky ne…",72,1763
"""https://www.csfd.cz/film/28158-zlatovlaska/prehled/""","""film""","""28158-zlatovlaska""","""Zlatovláska""","[""pohádka"", ""hudební""]","""Československo""",1973,74,"[""Vlasta Janečková""]","[""Vlasta Janečková""]","[""Petr Štěpánek"", ""Jorga Kotrbová"", … ""více""]","""České pohádky tak, jak je převyprávěla Božena Němcová a K. J. Erben patří k těm, k nimž se vracíme nejraději. Nejen jako čtenáři, ale také jako milovn…",70,13073
"""https://www.csfd.cz/film/108854-co-takhle-svatba-princi/prehled/""","""tv""","""108854-co-takhle-svatba-princi""","""Co takhle svatba, princi?""","[""pohádka"", ""hudební""]","""Československo""",1986,90,"[""Jiří Adamec""]","[""Jiří Chalupa""]","[""Miroslav Vladyka"", ""Ivana Andrlová"", … ""více""]","""Kdo by neznal jednu z nejznámějších filmových písniček ""Když se načančám"". Tuto a další písničky (např. ""Taková jsem já"", ""Princezny jdou si hajnout"" …",69,7403
"""https://www

In [53]:
musi_byt = [
    "Bohouš",
    "Dalskabáty, hříšná ves aneb Zapomenutý čert",
    "Chobotnice z II. patra",
    "Veselé Vánoce přejí chobotnice",
    "Nesmrtelná teta"
]

nebude = ['Shrek 2']

In [59]:
vanocni_final = vanocni_final.filter(
    (pl.col('pocet_hodnoceni') >= 26000) | (pl.col('nazev').is_in(musi_byt))
).filter(
    ~pl.col('nazev').is_in(nebude)
).sort(by='pocet_hodnoceni',descending=True)

vanocni_final

url,format,id,nazev,zanry,zeme,rok,delka,rezie,scenar,hraji,anotace,hodnoceni,pocet_hodnoceni
str,str,str,str,list[str],str,i64,i64,list[str],list[str],list[str],str,i64,i64
"""https://www.csfd.cz/film/4570-pelisky/prehled/""","""film""","""4570-pelisky""","""Pelíšky""","[""komedie"", ""drama""]","""Česko""",1999,116,"[""Jan Hřebejk""]","[""Petr Jarchovský""]","[""Miroslav Donutil"", ""Jiří Kodet"", … ""více""]","""Příběhy jedné historické generace - stárnoucích rodičů, dospívající mládeže a malých dětí. Děj je zasazen do konce šedesátých let - podzim 67 až léto …",91,117084
"""https://www.csfd.cz/film/14999-shrek/prehled/""","""film""","""14999-shrek""","""Shrek""","[""animovaný"", ""dobrodružný"", … ""komedie""]","""USA""",2001,90,"[""Vicky Jenson"", ""Andrew Adamson""]","[""Ted Elliott"", ""Terry Rossio"", … ""Roger S.H. Schulman""]","[""Mike Myers"", ""Eddie Murphy"", … ""více""]","""Shrek, zelený obr s tykadly mimozemšťanů, žije sice docela sám, ale šťastně ve svém domku na samotě u lesa až do chvíle, kdy je doslova přepaden nekon…",88,101749
"""https://www.csfd.cz/film/821-s-certy-nejsou-zerty/prehled/""","""film""","""821-s-certy-nejsou-zerty""","""S čerty nejsou žerty""","[""pohádka"", ""komedie""]","""Československo""",1984,91,"[""Hynek Bočan""]","[""Jiří Just"", ""Hynek Bočan""]","[""Luděk Munzar"", ""Vladimír Dlouhý"", … ""více""]","""Chamtivá macecha připraví svého nevlastního syna Petra o všechno, včetně dědictví po jeho otci. Petr se mezitím seznámí s dcerami místního knížete, kd…",89,80866
"""https://www.csfd.cz/film/1628-sam-doma/prehled/""","""film""","""1628-sam-doma""","""Sám doma""","[""rodinný"", ""komedie""]","""USA""",1990,103,"[""Chris Columbus""]","[""John Hughes""]","[""Macaulay Culkin"", ""Joe Pesci"", … ""více""]","""""Mám pocit, že jsme doma nechali něco důležitého,"" sdělí paní McCallisterová manželovi na palubě letadla směřujícího přes Atlantik do Paříže. To ještě…",86,80771
"""https://www.csfd.cz/film/43677-laska-nebeska/prehled/""","""film""","""43677-laska-nebeska""","""Láska nebeská""","[""komedie"", ""drama"", ""romantický""]","""Velká Británie / USA / Francie""",2003,135,"[""Richard Curtis""]","[""Richard Curtis""]","[""Bill Nighy"", ""Gregor Fisher"", … ""více""]","""Láska nebeská, romantická komedie ve stylu Čtyři svatby a jeden pohřeb, Notting Hill a Deník Bridget Jonesové, nás zavede do současného předvánočního …",85,76558
"""https://www.csfd.cz/film/9430-tri-orisky-pro-popelku/prehled/""","""film""","""9430-tri-orisky-pro-popelku""","""Tři oříšky pro Popelku""","[""pohádka""]","""Československo / Východní Německo""",1973,75,"[""Václav Vorlíček""]","[""František Pavlíček"", ""Václav Vorlíček""]","[""Libuše Šafránková"", ""Pavel Trávníček"", … ""více""]","""Popelka je milé hodné děvče bez matky a posléze i bez otce, který ji nechá napospas své druhé ženě a její dceři. Obě nemají Popelku rády a mimo jiné s…",86,73324
"""https://www.csfd.cz/film/6663-na-samote-u-lesa/prehled/""","""film""","""6663-na-samote-u-lesa""","""Na samotě u lesa""","[""komedie"", ""rodinný"", ""drama""]","""Československo""",1976,93,"[""Jiří Menzel""]","[""Zdeněk Svěrák"", ""Ladislav Smoljak""]","[""Josef Kemr"", ""Zdeněk Svěrák"", … ""více""]","""Hrdiny jsou manželé Oldřich a Věra Lavičkovi, kteří se svými dětmi Petříkem a Zuzankou odjíždějí do Loukova na chalupu svého známého, inženýra Radima …",88,66837
"""https://www.csfd.cz/film/10089-byl-jednou-jeden-kral/prehled/""","""film""","""10089-byl-jednou-jeden-kral""","""Byl jednou jeden král...""","[""pohádka"", ""komedie""]","""Československo""",1954,107,"[""Bořivoj Zeman""]","[""Jan Werich"", ""Jiří Brdečka"", ""Bořivoj Zeman""]","[""Jan Werich"", ""Vlasta Burian"", … ""více""]","""Byl jednou jeden král a ten si říkal Já První. Chtěl odevzdat vládu té ze tří dcer, která ho má nejraději. Drahomíra ho má ráda jako zlato, Zpěvanka j…",87,56191
"""https://www.csfd.cz/film/546-skala/prehled/""","""film""","""546-skala""","""Skála""","[""akční"", ""thriller"", ""dobrod

In [63]:
vanocni_final.sort(by="nazev").write_ndjson("data/vanocni_filmy_kompletni_metadata.json")

In [79]:
vanocni_final.sample(2)

url,format,id,nazev,zanry,zeme,rok,delka,rezie,scenar,hraji,anotace,hodnoceni,pocet_hodnoceni
str,str,str,str,list[str],str,i64,i64,list[str],list[str],list[str],str,i64,i64
"""https://www.csfd.cz/film/9926-prazdniny-v-rime/prehled/""","""film""","""9926-prazdniny-v-rime""","""Prázdniny v Římě""","[""komedie"", ""romantický""]","""USA""",1953,118,"[""William Wyler""]","[""Dalton Trumbo"", ""Ian McLellan Hunter"", ""John Dighton""]","[""Gregory Peck"", ""Audrey Hepburn"", … ""více""]","""Romantická komedie s A. Hepburn a G. Peckem v hlavních rolích. Prázdniny v Římě byly nominovány na 10 Oscarů a Audrey Hepburn získala Oscara za svou r…",88,26697
"""https://www.csfd.cz/film/10089-byl-jednou-jeden-kral/prehled/""","""film""","""10089-byl-jednou-jeden-kral""","""Byl jednou jeden král...""","[""pohádka"", ""komedie""]","""Československo""",1954,107,"[""Bořivoj Zeman""]","[""Jan Werich"", ""Jiří Brdečka"", ""Bořivoj Zeman""]","[""Jan Werich"", ""Vlasta Burian"", … ""více""]","""Byl jednou jeden král a ten si říkal Já První. Chtěl odevzdat vládu té ze tří dcer, která ho má nejraději. Drahomíra ho má ráda jako zlato, Zpěvanka j…",87,56191


In [95]:
df_pro_model = vanocni_final
neseznamove_sloupce = ['zeme','format']
seznamove_sloupce = ['zanry','rezie','scenar','hraji']
for s in neseznamove_sloupce:
    seznam_moznosti = df_pro_model.select(pl.col(s)).unique().to_series().to_list()
    for m in seznam_moznosti:
        df_pro_model = df_pro_model.with_columns(pl.lit(None).alias(f'{s}_{m}'))
        df_pro_model = df_pro_model.with_columns(pl.when(pl.col(s) == m).then(pl.lit(1)).otherwise(pl.col(f'{s}_{m}')).alias(f'{s}_{m}'))
for s in seznamove_sloupce:
    seznam_moznosti = df_pokus.explode(s).select(pl.col(s)).unique().to_series().to_list()
    for m in seznam_moznosti:
        df_pokus = df_pokus.with_columns(pl.lit(None).alias(f'{s}_{m}'))
        df_pokus = df_pokus.with_columns(pl.when(pl.col(s).list.contains(m)).then(pl.lit(1)).otherwise(pl.col(f'{s}_{m}')).alias(f'{s}_{m}'))
df_pokus = df_pokus.fill_null(0)

In [97]:
df_pokus.shape

(30, 1186)